In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ankita\\Desktop\\Personal_Project\\DL_Project\\chicken_disease_classification\\research'

In [3]:
os.chdir('../../')

In [4]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path

In [5]:
#configuration
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml, create_directories

In [6]:
class ConfiurationManger:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.configs = read_yaml(config_filepath)
        self.param = read_yaml(params_filepath)
        if not os.path.exists(self.configs.artifact_root):
            create_directories([self.configs.artifact_root])
     
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.configs.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir), 
            Path(config.tensorboard_root_log_dir)
        ])
            
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)                        
        )
        return prepare_callback_config

In [7]:
import os
from pathlib import Path
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [8]:
#component
class PrepareCallback:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config = config
    
    @property
    def create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at{timestamp}",
            
        )
        return tf.keras.callbacks.TensorBoard(log_dir= tb_running_log_dir)
    @property
    def create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )
        
    def get_tb_ckpt_callbacks(self):
        return[
            self.create_tb_callbacks,
            self.create_ckpt_callbacks
            ]
        

In [9]:
#pipeline

try:
    config = ConfiurationManger()
    prepare_callback_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config= prepare_callback_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e
    

[2024-09-07 17:06:50,782: INFO : common: yaml file: chicken_disease_classification\config\config.yaml loaded successfully]
[2024-09-07 17:06:50,785: INFO : common: yaml file: chicken_disease_classification\params.yaml loaded successfully]
[2024-09-07 17:06:50,786: INFO : common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-09-07 17:06:50,787: INFO : common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
